In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
import sklearn.calibration
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
# plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from math import floor
from copy import deepcopy
from sklearn.metrics import confusion_matrix
import config
import models
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
import neural_networks
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F
import interpret
import matplotlib.pyplot as plt
from functools import partial
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# prepare data

In [6]:
dsets = ['vps4_snf7', 'vps4_snf7___key=mt']
splits = ['train', 'test']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
dfs, feat_names = data.load_dfs_for_lstm(dsets=dsets,
                                         splits=splits,
                                         filter_hotspots=True,
                                         filter_short=False,
                                         hotspots_threshold=25,
                                         meta=meta,
                                         normalize=False)


print('type', type(dfs))
# load model
p = 1
for k in dfs:
    print(*k, dfs[k].shape)
df = dfs[list(dfs.keys())[0]]
X = df[feat_names[:p]]
y = df['y_consec_thresh']

100%|██████████| 2/2 [00:10<00:00,  5.07s/it]

type <class 'dict'>
vps4_snf7 train (8962, 90)
vps4_snf7 test (2951, 90)
vps4_snf7___key=mt train (3378, 90)
vps4_snf7___key=mt test (1210, 90)


**summarize dsets**

In [8]:
ds = dfs
vals = {} # key is dataset, Partition
for k in sorted(ds, key=lambda kv: (kv[0], kv[1])):
    d = ds[k]
    d.short = d.lifetime <= 3
    # print(k, d.lifetime.min())
    y = d['y_consec_thresh']
    feats = ['Total clean tracks',
             'Difficult tracks', 'Difficult valid events', 
             'Short tracks', 'Short valid events',]
    vals[k] = [
        d.shape[0],
        (~d.short).sum(), y[~d.short].sum(),
        d.short.sum(), y[d.short].sum()
    ]
    vals[k] = [int(x) for x in vals[k]]
#     print(feats, vals)
df = pd.DataFrame.from_dict(vals).transpose().round(decimals=0)
df.columns = feats
df.index = df.index.set_names(['Dataset', 'Partition'])
print(df.to_latex(index=True, index_names=True))

\begin{tabular}{llrrrrr}
\toprule
                   &       &  Total clean tracks &  Difficult tracks &  Difficult valid events &  Short tracks &  Short valid events \\
Dataset & Partition &                     &                   &                         &               &                     \\
\midrule
vps4\_snf7 & test &                2951 &              1922 &                     360 &          1029 &                  77 \\
                   & train &                8962 &              7420 &                    1505 &          1542 &                 130 \\
vps4\_snf7\_\_\_key=mt & test &                1210 &               999 &                     186 &           211 &                  29 \\
                   & train &                3378 &              2734 &                     601 &           644 &                 107 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_978228/1617068531.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=True, index_names=True))


In [9]:
df

Total clean tracks  Difficult tracks  \
Dataset            Partition                                         
vps4_snf7          test                     2951              1922   
                   train                    8962              7420   
vps4_snf7___key=mt test                     1210               999   
                   train                    3378              2734   

                              Difficult valid events  Short tracks  \
Dataset            Partition                                         
vps4_snf7          test                          360          1029   
                   train                        1505          1542   
vps4_snf7___key=mt test                          186           211   
                   train                         601           644   

                              Short valid events  
Dataset            Partition                      
vps4_snf7          test                       77  
                   train                     130  
vps4_snf7___key=mt test                       29  
                   train                     107

In [14]:
splits = ['train', 'test']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
length = 40
#     padding = 'end'
feat_name = 'X_same_length_extended_normalized' # include buffer X_same_length_normalized
outcome = 'Y_sig_mean_normalized'

df_fulls = []
for i, dsets in enumerate([['vps4_snf7'], ['vps4_snf7___key=mt']]):

    dfs, feat_names = data.load_dfs_for_lstm(dsets=dsets,
                                                splits=splits,
                                                filter_hotspots=True,
                                                filter_short=False,
                                                lifetime_threshold=None,
                                                hotspots_threshold=25,
                                                meta=meta,
                                                normalize=False)
    df_full = pd.concat([
        dfs[(k, s)]
        for (k, s) in dfs
        if s == 'train'
    ])
    df_full['mt'] = i
    df_fulls.append(df_full)

df_full = pd.concat(df_fulls)


100%|██████████| 1/1 [00:02<00:00,  3.00s/it]
